In [1]:
import pandas as pd
import numpy as np

In [2]:
path_data = "https://github.com/DavidBreuer/ifsc-analysis/raw/main/ifsc_Boulder.xlsx"

df_raw = pd.read_excel(path_data)

In [3]:
df = (
    df_raw
    # Get rid of columns we don't need
    .drop(columns=['Unique', 'Discipline', 'Number', 'Group'])
    .drop(columns=df_raw.filter(regex='Route|Run').columns)
    # Capitalize climber names consistently
    .assign(Name=lambda x: x['Name'].str.title())
    # "Unpivot" so it's one row per climber-problem
    # I'm treating tops and zones as separate problems even though there's obviously a correlation
    .melt(id_vars=['Year', 'Competition', 'Gender', 'Level', 'Name', 'Country'],
          value_vars=['Top1', 'Top2', 'Top3', 'Top4', 'Top5', 'Zone1', 'Zone2', 'Zone3', 'Zone4', 'Zone5'],
          var_name='problem', value_name='attempts')
    .dropna(subset=['attempts'])
)

In [4]:
# Keep completed boulders
grouped = df.groupby(['Competition', 'Level', 'problem'])
df = grouped.filter(lambda x: np.isfinite(x['attempts']).any())


# Failure set to max attempts
df = df.assign(max_attempts=lambda x: grouped['attempts'].transform('max'))
df['attempts'] = np.where(df['attempts'].isna(), df['max_attempts'], df['attempts'])


# Survival model features
df = df.assign(
    status=lambda x: x['attempts'].notna(),
    time=lambda x: np.where(x['attempts'].isna(), x['max_attempts'], x['attempts'])
)


# Keep Climbers with lots of data; "Other" as replacement
data_threshold = 1000
climber_counts = df['Name'].value_counts()
df['climber'] = np.where(df['Name'].map(climber_counts) >= data_threshold, df['Name'], 'Other')
df['climber'] = pd.Categorical(df['climber'], categories=['Other'] + [x for x in df['climber'].unique() if x != 'Other'])


# Data Cleaning Analysis
m_count, w_count = df[(df['Gender']=="M") & (df['climber']!="Other")].climber.nunique(), df[(df['Gender']=="W") & (df['climber']!="Other")].climber.nunique()
m_total_count, w_total_count = df_raw[df_raw['Gender']=='M'].Name.nunique(), df_raw[df_raw['Gender']=='W'].Name.nunique()

print(f'There are {m_count} male contestants with over {data_threshold} data points out of {m_total_count}')
print(f'There are {w_count} female contestans with over {data_threshold} data points out of {w_total_count}')

m_contestants = df[(df['Gender']=="M") & (df['climber']!="Other")].Name.unique()
w_contestants = df[(df['Gender']=="W") & (df['climber']!="Other")].Name.unique()

print(f'The male contestans are {", ".join(m_contestants)}')
print(f'The female contestans are {", ".join(w_contestants)}')

There are 11 male contestants with over 1000 data points out of 1424
There are 7 female contestans with over 1000 data points out of 1007
The male contestans are Dmitrii Sharafutdinov, Kilian Fischhuber, Rustam Gelmanov, Guillaume Glairon Mondet, Jernej Kruder, Tsukuru Hori, Sean Mccoll, Aleksei Rubtsov, Jan Hojer, Kokoro Fujii, Tomoa Narasaki
The female contestans are Akiyo Noguchi, Katharina Saurwein, Anna Stöhr, Melissa Le Neve, Petra Klingler, Shauna Coxsey, Miho Nonaka


In [5]:
df[df.Gender=="M"].to_csv('./data/men_data_1000.csv', index=False)
df[df.Gender=="W"].to_csv('./data/women_data_1000.csv', index=False)